In [7]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pandas as pd
import numpy as np

#Move indicator into its own Class so that will show up in output file. 
#Reference: https://www.backtrader.com/docu/inddev.html
class RSI_Oversold(bt.Indicator):
    lines = ('oversold',)
    
    params = (
        ('level', 30),    
    )

    def __init__(self):
        self.lines.oversold = bt.indicators.RSI(self.data) < self.p.level

class Bollinger_Breakdown(bt.Indicator):
    lines = ('breakdown',)
    
    def __init__(self):
        self.lines.breakdown = self.data < bt.indicators.BBands(self.data, period=20).lines.bot        
        
class Ultimate_Oscillator(bt.Indicator):
    lines = ('uo',)
     
    params = (
        ('level', 30),    
    )
    
    # Add custom indicator logic here...currently returns value of indicator
    def __init__(self):
        self.lines.uo = bt.indicators.UltimateOscillator(self.data).lines.uo          
        
    
# Create a Strategy
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ("symbols", None),
        ("max_days_in_trade", 5)
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        symbols = self.p.symbols
        
        # Keep a reference to the close and forward lagged close prices as well as returns
        self.stock = self.datas[symbols.index("stockpx")]
        self.stock_px = self.stock.close
        self.stock_px_enter = self.datas[symbols.index("stockpx_lag_1")].close
        self.stock_px_exit = self.datas[symbols.index("stockpx_lag_%d" % self.p.max_days_in_trade)].close
        self.stock_returns = self.stock_px / self.stock_px(-1) - 1
        
        # Add indicators
        # Use this instead of below to be able to see full indicator (not just RSI) in output file
        self.rsi_oversold = RSI_Oversold(self.stock_px)
        #self.rsi_oversold = bt.indicators.RSI(self.stock_px) <30
        
        self.ulti_osc = Ultimate_Oscillator(self.stock) #pass in stock not just stock_px as indicator also requires high/low prices
        self.bollinger_breakdown = Bollinger_Breakdown(self.stock_px)
        
        self.in_trade = False
        self.days_in_trade = 0
        
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.stock_px[0])
        
        size = self.position.size
        
        if(not size):
            #if self.rsi_oversold[0] > 0:
            if self.bollinger_breakdown[0] > 0:
                # replicate straddle payout less premium
                print("Indicator triggered...entering position")
                print('Exit price %.2f enter price %.2f' % (self.stock_px_exit[0], self.stock_px_enter[0]))
                if self.stock_px_exit[0] > self.stock_px_enter[0]:
                    print("Exit price will be higher than enter price. Buy!")
                    self.buy(exectype=bt.Order.Close)
                else:
                    print("Exit price will be lower than enter price. Sell!")
                    self.sell(exectype=bt.Order.Close)
                self.days_in_trade = 1
        else:
            self.days_in_trade += 1
            print("In position %d days" % self.days_in_trade)
            if self.days_in_trade == self.p.max_days_in_trade:
                print("Max days to hold trade reached...closing position")
                self.close(exectype=bt.Order.Close)
                self.days_in_trade = 0
                

if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    
    asset = 'MSFT'
    n_lags = 6 
    
    # Read in stock and vol data 
    datapath_vol = os.path.join('../../../datas/%s.csv' % asset) 
    datapath_stock = os.path.join('../../../datas/%s-2010-2018.csv' % asset)
    data_vol = pd.read_csv(datapath_vol, parse_dates=True,index_col=0)
    data_stock = pd.read_csv(datapath_stock, parse_dates=True,index_col=0)
    
    # Lag and merge data
    for n in np.arange(1, n_lags):
        data_stock['stockpx_lag_%d' % n ] = data_stock['Close'].shift(-n)  
    data_stock = data_stock.dropna()
    
    mergedDf = pd.concat([data_vol,data_stock], axis=1,join='inner')
    data_vol = data_vol.ix[data_vol.index & mergedDf.index]
    data_stock = data_stock.ix[data_stock.index & mergedDf.index]
    
    # Add stock price data
    cerebro.adddata(bt.feeds.PandasData(dataname=data_stock[['Close', 'High', 'Low', 'Volume']]), name='stockpx')
    
    # Add lagged stock prices to be able to replicate straddle payout
    symbol_cols = [ 'stockpx_lag_%d' % n for n in np.arange(1, n_lags)] 
    for i, symbol in enumerate(symbol_cols):
        data = data_stock[[symbol]]
        data.columns = ["Close"]
        cerebro.adddata(bt.feeds.PandasData(dataname=data), name=symbol)
    
    # Add straddle premium
    data = data_vol[['straddle']]
    data.columns = ["Close"]
    cerebro.adddata(bt.feeds.PandasData(dataname=data), name='straddle')
    
    # Add a strategy
    symbol_cols = ['stockpx'] + symbol_cols + ['straddle']
    cerebro.addstrategy(TestStrategy, symbols=symbol_cols)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='straddle_indicators.csv',csv=True)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

/home/project/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:124: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/project/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:125: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


Starting Portfolio Value: 100000.00
2010-02-12, Close, 27.93
2010-02-16, Close, 28.35
2010-02-17, Close, 28.59
2010-02-18, Close, 28.97
2010-02-19, Close, 28.77
2010-02-22, Close, 28.73
2010-02-23, Close, 28.33
2010-02-24, Close, 28.63
2010-02-25, Close, 28.60
2010-02-26, Close, 28.67
2010-03-01, Close, 29.02
2010-03-02, Close, 28.46
2010-03-03, Close, 28.46
2010-03-04, Close, 28.63
2010-03-05, Close, 28.59
2010-03-08, Close, 28.63
2010-03-09, Close, 28.80
2010-03-10, Close, 28.97
2010-03-11, Close, 29.18
2010-03-12, Close, 29.27
2010-03-15, Close, 29.29
2010-03-16, Close, 29.37
2010-03-17, Close, 29.63
2010-03-18, Close, 29.61
2010-03-19, Close, 29.59
2010-03-22, Close, 29.60
2010-03-23, Close, 29.88
2010-03-24, Close, 29.65
2010-03-25, Close, 30.01
2010-03-26, Close, 29.66
2010-03-29, Close, 29.59
2010-03-30, Close, 29.77
2010-03-31, Close, 29.29
2010-04-01, Close, 29.16
2010-04-05, Close, 29.27
2010-04-06, Close, 29.32
2010-04-07, Close, 29.35
2010-04-08, Close, 29.92
2010-04-09, Cl